In [1]:
import sklearn as sk
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import os


In [5]:
num_clusts = range(40)


In [13]:
X = pd.read_csv(os.getcwd() + '/top5_chars.csv')
X.index = X.iloc[:,0]
X.drop('Unnamed: 0',axis=1,inplace=True)
X.drop(['cursewords','FK','0'],axis=1,inplace=True)
X.head()

,0,tot_feel,tot_fill,tot_hw,stopwords,polarity,uniqwords
Unnamed: 0,,,,,,,
KAT____10-Things-I-Hate-About-You_script,"leave it i said , leave it ! why did n't we ju...",18.0,0.0,87.0,97.0,-0.9984,603.0
PATRICK____10-Things-I-Hate-About-You_script,i missed you . it was a bratwurst . i was eati...,6.0,13.0,60.0,86.0,-0.9952,465.0
BIANCA____10-Things-I-Hate-About-You_script,did you change your hair ? you might wan na th...,13.0,9.0,75.0,85.0,0.9968,418.0
CAMERON____10-Things-I-Hate-About-You_script,"i do n't think so , ma'am so they tell me ... ...",8.0,8.0,58.0,71.0,0.9951,316.0
MICHAEL____10-Things-I-Hate-About-You_script,you the new guy ? c'mon . i 'm supposed to giv...,7.0,3.0,46.0,83.0,0.9984,419.0


In [14]:
#so, we will actually have to break these guys down into senteces... 
# so I will have to do the sentence tokenizer :) 
def jacardian_distance(document_1_data, document_2_data):

    
    #for key in processed_article_hash.keys(): 
    #art_uniq_ws[key] = []
    #for sent in processed_article_hash[key]:
    #    art_uniq_ws[key].extend(sent)
    #    art_uniq_ws[key] = list(set(art_uniq_ws[key]))
    d1_uniqs = []
    for sent in document_1_data:
        d1_uniqs.extend(sent)
        d1_uniqs = list(set(d1_uniqs))
    d2_uniqs = []
    for sent in document_2_data:
        d2_uniqs.extend(sent)
        d2_uniqs = list(set(d2_uniqs))
        
    words_in_doc_1_not_in_doc_2 = [value for value in d1_uniqs if value not in d2_uniqs]
    words_in_doc_2_not_in_doc_1 = [value for value in d2_uniqs if value not in d1_uniqs]
    words_in_both_doc_1_and_doc_2 = [value for value in d1_uniqs if value in d2_uniqs]
    
    
    
    #overlap = [value for value in d1_uniqs if value in d2_uniqs] 
    #in1Not2 = [value for value in d1_uniqs if value not in d2_uniqs]
    #in2Not1 = [value for value in d2_uniqs if value not in d1_uniqs]
   # print(len(in1Not2))
    #print(len(in2Not1))
    #print(len(sv))
   # print(len(sv)+len(in2Not1)+len(in1Not2))

   # sv = d2_uniqs + d1_uniqs
   # sv = list(set(sv))
   # print(len(overlap))
    
    
    
    jacardian = len(words_in_both_doc_1_and_doc_2)/(len(words_in_both_doc_1_and_doc_2) + len(words_in_doc_1_not_in_doc_2) + len(words_in_doc_2_not_in_doc_1)) # divide the counts appropiately
    
    return jacardian
    
    
    
#Cosine Similarity: Here we want to create vector representations for each document. Specifically, we want to come up with a vector that is based on the list of all words that occur across both documents. Then for each document we will create a vector that includes the counts of the number of time a word occurs in the document.

#So if the document 1 is: "the ship sails at midnight" and document 2 is: "the crow flies at noon." We would be creating a vector like: [the, ship, sails, at, midnight, crow, flies, noon]. Then we would calculate the values of the vector for each document. For document 1: [1,1,1,1,1,0,0,0] and for document 2: [1,0,0,1,0,1,1,1]. With these two vectors we would simply take the dot product and that would provide the cosine similarity. 

def cosine_similarity(document_1_data, document_2_data):
    document_vector_word_index = [] # here fill this with an ordered list of all the unique words across both documents
    d1_uniqs = []
    d1_nun = []
    for sent in document_1_data:
        d1_uniqs.extend(sent)
        d1_nun.extend(sent)
        d1_uniqs = list(set(d1_uniqs))
    d2_uniqs = []
    d2_nun = []
    for sent in document_2_data:
        d2_uniqs.extend(sent)
        d2_uniqs = list(set(d2_uniqs))
        d2_nun.extend(sent)

    
    all_words = d1_nun+d2_nun   

    overlap = d1_uniqs + d2_uniqs 
    # so this is all of the unique words across the two documents
    document_vector_word_index += list(set(overlap))
    

    document_1_vector = list(np.zeros(len(document_vector_word_index)))  # fill in the array with the frequency of the words in the document
    document_2_vector = list(np.zeros(len(document_vector_word_index))) # fill in the array with the frequency of the words in the document

    for wOrd in d1_nun:
        document_1_vector[document_vector_word_index.index(wOrd)] +=1
    
    for wOrd in d2_nun:
        document_2_vector[document_vector_word_index.index(wOrd)] +=1
    
    d1_norm = (np.sum(np.array(document_1_vector)**2))**(1/2)
    d2_norm = (np.sum(np.array(document_2_vector)**2))**(1/2)
    
    cosSim = np.dot(document_1_vector,document_2_vector)/(d1_norm*d2_norm)


    
    return (cosSim)
#document_1_vector,document_2_vector,document_vector_word_index, are the things I tried to return in testing
#dot_product_of_two_document_vectors # you can refer to the numpy information on how to calculate the dot product of vectors

In [ ]:
# create a variable to store your table data... you could use a hash or some other data structure. 
# We just want it to identify which document is being compared to which other document.

data_structure_for_jacard_similarity = []
data_structure_for_cosine_similarity = []
# so I think what I'm ACTUALLY going to do here 
# is just append a new column
# and then say the column is .... 
# so we are comparing each character to every OTHER character (including themselves)
for char_1_index in char_series.index:
    for char_2_index in char_series.index:
        # we have the nested for loops as one way to compare each document to each other document
        data_structure_for_jacard_similarity.append(jacardian_distance(char_series[char_1_index], char_series[char_2_index]))
        data_structure_for_cosine_similarity.append(cosine_similarity(char_series[char_1_index], char_series[char_2_index]))

# finally, find some way to present this data back. Either as a straight table or a heatmap.
# so... yeah.... I can probably keep the rows the same as what they were... TECHNICALLY.... 
colz = char_series.index
rowz = char_series.index


for movChar in char_series_index:
    

In [15]:
pd.Series(np.nan,index = char_series.index)

NameError: name 'char_series' is not defined